# Focused Ion Beam Molecular Dynamics (fibmd) Tool

In [ ]:
from IPython.display import display, clear_output, Javascript
import ipywidgets as widgets
import numpy as np
from string import Template
import hublib.use
import py3Dmol
from IPython.display import FileLink, FileLinks
from hublib.ui import Submit
#from hublib.cmd import runCommand

In [ ]:
%use openmpi-1.6.3-gnu-4.7.2

# Creating widgets for input

In [ ]:
#title = widgets.Label(value="FIBMD Tool", width=10)

box_layout = widgets.Layout(display='flex',
                            justify_content='center')
text_layout = widgets.Layout(width = '150px')
text_layout1 = widgets.Layout(width = '50px')
text_layout2 = widgets.Layout(width ='100px',
                              position = 'right')

keVin = widgets.BoundedFloatText(
    value=2000,
    min=1000,
    max=60000,
    step=100,
    layout=text_layout2
)

xdimin = widgets.BoundedFloatText(
    value=5,
    min=4,
    max=40,
    step=1,
    layout=text_layout1
)

ydimin = widgets.BoundedFloatText(
    value=5,
    min=4,
    max=40,
    step=1,
    layout=text_layout1
)

zdimin = widgets.BoundedFloatText(
    value=2,
    min=1,
    max=40,
    step=1,
    layout=text_layout1
)

pin = widgets.BoundedIntText(
    value = 1,
    min = 1,
    max = 10,
    step = 1,
    layout=text_layout2
)

tempin = widgets.FloatText(
    value = 300.0,
    step = 10.0,
    layout = text_layout2
)

nin = widgets.BoundedIntText(
    value = 1,
    min = 1,
    max = 100,
    step = 1,
    layout=text_layout2
)

ain  = widgets.BoundedIntText(
    value = 500,
    min = 0,
    max = 1e6,
    step = 100,
    layout=text_layout2
)

dtin = widgets.BoundedFloatText(
    value = 1,
    step = 0.1,
    layout=text_layout2,
    min = 0.1
)

phizin = widgets.FloatText(
    value = 0.0,
    step = 1,
    layout=text_layout2
)

phixyin = widgets.FloatText(
    value = 0.0,
    step = 1,
    layout=text_layout2
)

fin = widgets.FloatText(
    value = 2,
    layout=text_layout2
)

dimlabel = widgets.Label(" Target dimensions, nm",layout=box_layout)

xlabel = widgets.Label("    X:", position='right')
ylabel = widgets.Label("    Y:", position='right')
zlabel = widgets.Label("    Z:", position='right')
xbox = widgets.HBox([xlabel,xdimin],layout = text_layout)
ybox = widgets.HBox([ylabel,ydimin],layout = text_layout)
zbox = widgets.HBox([zlabel,zdimin],layout = text_layout)

keVlabel = widgets.Label("Ion Energy (eV):")
proclabel = widgets.Label("# of Processors:")
templabel = widgets.Label("Silicon Temperature (K):")
ionlabel = widgets.Label("# of Ions to fire:")
outlabel = widgets.Label(".xyz output frequency:")
dtlabel = widgets.Label("Time between ion impacts in ps:")
anglezlabel = widgets.Label("Beam Angle $\\theta$ (from z-axis):")
anglexylabel = widgets.Label("Beam Angle $\Phi$ (from x-axis):")
fwhmlabel = widgets.Label("FWHM of ion beam in nm:")
beamlabel = widgets.Label("Effective Beam Current (nA): ")
sizelabel = widgets.Label("Estimated Total File Size (MB)")
fluxlabel = widgets.Label("$4\sigma$ Surface Flux (ions/cm$^2$ps): ")

dimin = widgets.HBox([xbox,ybox,zbox])
vlabels = widgets.VBox([keVlabel, proclabel, templabel, ionlabel, outlabel, dtlabel, anglezlabel, anglexylabel, fwhmlabel])
vinput = widgets.VBox([keVin, pin, tempin, nin, ain, dtin, phizin, phixyin, fin])

inputs2 = widgets.HBox([vlabels,vinput])

#silicon lattice constant
silat = 0.5431

## dynamically updating boxes for current, estimated file size, flux
BeamCurrent = widgets.Text(
    value = '%0.3f'%(1.602e-19/(dtin.value*1e-12)*1e9),
    layout = text_layout2,
    position = 'right'
)

SurfFlux = widgets.Text(
    value = '%1.4E'%(1/dtin.value/((fin.value/2.355*4*1e-7)**2*np.pi/4)
    ),
    layout = text_layout2,
        position = 'right'
)

    #6.44e-5 is estimate of MB per atom per snapshot
    #floors of dimin are unit cells, 8 is atoms per unit cell
    #.1/.02e-3 is estimating number of timesteps in fast regime (lasting .1 ps, with .02e-3 step size)
    #(dtin.value-.1)/.2e-3 is # of timesteps during the rest of the time
    #ain.value/10 and ain.value normalize total timesteps by output frequency in each case
    #whole process happens nin.value times
EstimatedSize = widgets.Text( 
    value = '%0.3f'%(6.5e-05*(np.floor(xdimin.value/silat) * 
                      np.floor(ydimin.value/silat) * 
                      np.floor(zdimin.value/silat))*8 *
             (.1/.02e-3 /(ain.value/10) + (dtin.value-.1)/.2e-3/(ain.value))*nin.value
),
        layout = text_layout2,
        position = 'right'
)

vlabels2 = widgets.VBox([beamlabel,fluxlabel,sizelabel])
outlabels = widgets.VBox([BeamCurrent,SurfFlux,EstimatedSize])
outlabels2 = widgets.HBox([vlabels2,outlabels])

In [ ]:
input1 = widgets.VBox([dimlabel,dimin,inputs2],layout=widgets.Layout(border='solid'))
display(input1)
display(outlabels2)
#display(widgets.HBox([beamlabel, BeamCurrent]))
#display(widgets.HBox([sizelabel, EstimatedSize]))
#display(widgets.HBox([fluxlabel, SurfFlux]))

In [ ]:
!touch out.log data/mdrun2.xyz data/3dmol2.xyz
#file download links
outfile = FileLink("out.log")
xyzfile = FileLink("data/mdrun2.xyz")
ovitofile = FileLink("data/OvitoTemplate.ovito")
display(outfile,xyzfile,ovitofile)

# Function Definitions

In [ ]:
##For determining how to distribute processor load in domain
#list of prime factors for some number 'n'
def prime_factors(n):
    i = 2
    factors = []
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
            factors.append(i)
    if n > 1:
        factors.append(n)
    return factors

#determine 3 integer factors of nc that are closest together
def primefactors3(nc):
    a=prime_factors(nc)
    z = np.ones(3,dtype=np.int8)
    zb = []

    if len(a) < 3:
        z[3-len(a):] = a
        b = range(0)
    else:
        z = z = a[len(a)-3:]
        b = range(len(a)-4,-1,-1)

    for i in b:
        indmin = np.argmin(z)
        z[indmin]=z[indmin]*a[i]    

    zb.append(z[np.argmax(z)])
    zb.append(int(np.sum(z)-z[np.argmin(z)]-z[np.argmax(z)]))
    zb.append(z[np.argmin(z)])
    return zb

#Create the input file from siga.in.template
def get_template():
    procs = primefactors3(pin.value)
    if ain.value == 0:
        atmoutin1 = -1
        atmoutin2 = -1
    else:
        atmoutin1 = int(ain.value/10)
        atmoutin2 = ain.value
        dims = [int(xdimin.value/silat), int(ydimin.value/silat), int(zdimin.value/silat)]
        
    inputs = list([str(keVin.value),str(dims[0]),str(dims[1]),str(dims[2]),
                         str(procs[0]),str(procs[1]),str(procs[2]),str(tempin.value),str(nin.value),str(atmoutin1),str(atmoutin2),
                         str(dtin.value*1e-12),str(fin.value),str(phizin.value),str(phixyin.value)])
    
    tags = list(['keV','xdim','ydim','zdim',
                 'procsx','procsy','procsz','Ttar1','nlj','atmout1','atmout2',
                 'dtion','fwhm','phiz','phixy'])

    input_dict = dict(zip(tags, inputs))
    temp_contents = open('siga.in.template').read()
    tempstr = Template(temp_contents)
    return tempstr.substitute(input_dict)


##Callback functions 
def BC_cb(change):
    BeamCurrent.value = str(1.602e-19/(dtin.value*1e-12)*1e9) #atoms/ps to nA
    
def ES_cb(change):
    if ain.value <= 0:
        EstimatedSize.value = '0'
    else:
        EstimatedSize.value = '%0.3f'%(6.5e-05*(np.floor(xdimin.value/silat) * 
                      np.floor(ydimin.value/silat) * 
                      np.floor(zdimin.value/silat))*8*
             (.1/.02e-3 /(ain.value/10) + (dtin.value-.1)/.2e-3/(ain.value))*nin.value)

def SF_cb(change):
        SurfFlux.value = '%1.4E'%(1/dtin.value/((fin.value/2.355*4*1e-7)**2*np.pi/4))
        
        
#button click run function
def my_start(s):
    rname = s.make_rname(pin.value)
    !rm siga.in
    with open('siga.in', 'w') as tfile:
        tfile.write(get_template())

    # run locally for now, also split output to out.log and stdout
    submit_str = '--local mpirun -np %i bin/mdrun2 | tee out.log'%(pin.value)
    s.run(rname, submit_str)
    
#button click for updating the visualization window
def updateviewer(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))
UV = widgets.Button(button_style='info',description="Update Viewer")
UV.on_click(updateviewer)


## Observers, for dynamic input/output box updates
dtin.observe(BC_cb,names = 'value')
dtin.observe(ES_cb,names = 'value')
dtin.observe(SF_cb,names = 'value')
ain.observe(ES_cb,names = 'value')
nin.observe(ES_cb,names = 'value')
xdimin.observe(ES_cb,names = 'value')
ydimin.observe(ES_cb,names = 'value')
zdimin.observe(ES_cb,names = 'value')
fin.observe(SF_cb,names = 'value')

#definition for the visualization window
p = py3Dmol.view(width=520,height=480)

In [ ]:
test = Submit(start_func=my_start, cachename='SubmitTest')
test

In [ ]:
display(UV)

In [ ]:
#3dmol viewer
#display(widgets.Text(value = 'control+click to slide'))
#print('\n')
file = 'data/3dmol2.xyz'
xyzout = open(file,'r').read()
p.clear()
p.addModel(xyzout,'xyz')
p.setStyle({'sphere':{}})
p.setBackgroundColor('0xeeeeee')
#p.clear()
#p.zoom()
p.render()
#display(widgets.Text(value = '\n'))

